In [1]:
# import relevant commands
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [38]:
# Reading the FractureGrowth dateset Exploring the growth
growth = pd.read_csv('FractureGrowth.csv')
growth.head()

,Young's Modulus (GPa),Poisson's ratio,Tensile Strength (MPa),Compressive strength (MPa),Density (g/cm3),Friction coefficient,Inclination angle,X,Y,Fracture Growth
0,2.9,0.44,75,139,1.18,0.53,0,1.752987,5.035553,0
1,2.9,0.44,75,139,1.18,0.53,0,3.200039,5.402463,1
2,2.9,0.44,75,139,1.18,0.53,0,3.279343,4.390618,1
3,2.9,0.44,75,139,1.18,0.53,0,3.098813,3.604574,1
4,2.9,0.44,75,139,1.18,0.53,0,3.384115,5.025376,0


In [5]:
growth.describe()

,Young's Modulus (GPa),Poisson's ratio,Tensile Strength (MPa),Compressive strength (MPa),Density (g/cm3),Friction coefficient,Inclination angle,X,Y,Fracture Growth
count,1.100000e+01,1.100000e+01,11.0,11.0,11.00,1.100000e+01,11.0,11.000000,11.000000,11.000000
mean,2.900000e+00,4.400000e-01,75.0,139.0,1.18,5.300000e-01,0.0,2.948121,4.849656,0.727273
std,9.315294e-16,5.822059e-17,0.0,0.0,0.00,1.164412e-16,0.0,0.438360,1.659039,0.467099
min,2.900000e+00,4.400000e-01,75.0,139.0,1.18,5.300000e-01,0.0,1.752987,2.259991,0.000000
25%,2.900000e+00,4.400000e-01,75.0,139.0,1.18,5.300000e-01,0.0,2.950267,3.807855,0.500000
50%,2.900000e+00,4.400000e-01,75.0,139.0,1.18,5.300000e-01,0.0,3.041936,5.025376,1.000000
75%,2.900000e+00,4.400000e-01,75.0,139.0,1.18,5.300000e-01,0.0,3.149426,5.734579,1.000000
max,2.900000e+00,4.400000e-01,75.0,139.0,1.18,5.300000e-01,0.0,3.384115,7.773352,1.000000


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
features = titanic.drop('Fracture Growth', axis=1)
labels = titanic['Fracture Growth']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [8]:
for dataset in(y_train, y_val, y_test):
    print(round(len(dataset) / len(labels), 2))

0.55
0.27
0.18


In [9]:
print(len(labels), len(y_train), len(y_val), len(y_test))

11 6 3 2


In [11]:
# Write out train, test, validation data
X_train.to_csv('Fracture_train_features.csv' , index=False)
X_val.to_csv('Fracture_val_features.csv' , index=False)
X_test.to_csv('Fracture_test_features.csv' , index=False)

y_train.to_csv('Fracture_train_labels.csv' , index=False)
y_val.to_csv('Fracture_val_labels.csv' , index=False)
y_test.to_csv('Fracture_test_labels.csv' , index=False)

In [12]:
# Logistic Regression Model
# Gives potential hyperparameters we could tune. We would tune the one which has the largest impact
# The C hyperparameter is regularisation parameter is logistic reg that controls how closely model fits to training data
# More value of C, less regularization and classification good, if less C then more regularization and underfitting
from sklearn.linear_model import LogisticRegression

LogisticRegression()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

tr_features = pd.read_csv('Fracture_train_features.csv')
tr_labels = pd.read_csv('Fracture_train_labels.csv', header=None)

In [20]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))
    
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean,3), round(std * 2, 3), params))

In [24]:
# Logistic Regression with 4-fold cross validation and tuning the hyperparameter C
# Shows that model underfits when low C & high Regularization & less accuracy; C=1 best accuracy; model overfits when high C & low Reg
# 7 hyperparameters and 4 cross val so 7*4 = 28 individual models
# Best is 75% as below
lr = LogisticRegression()
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
}

cv = GridSearchCV(lr, parameters, cv=4)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

C:\Users\shasw\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


BEST PARAMS: {'C': 0.001}

0.75 (+/-0.5) for {'C': 0.001}
0.75 (+/-0.5) for {'C': 0.01}
0.75 (+/-0.5) for {'C': 0.1}
0.375 (+/-0.829) for {'C': 1}
0.375 (+/-0.829) for {'C': 10}
0.5 (+/-0.707) for {'C': 100}
0.5 (+/-0.707) for {'C': 1000}


In [25]:
cv.best_estimator_

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
# Write out pickled model
joblib.dump(cv.best_estimator_,'FGLR_model.pkl')

['FGLR_model.pkl']

In [33]:
# Evaluate results on the validation set
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time

val_features = pd.read_csv('Fracture_val_features.csv')
val_labels = pd.read_csv('Fracture_val_labels.csv', header=None)

te_features = pd.read_csv('Fracture_test_features.csv')
te_labels = pd.read_csv('Fracture_test_labels.csv', header=None)

In [34]:
models = {}

for mdl in ['FGLR']:
    models[mdl] = joblib.load('{}_model.pkl'.format(mdl))

In [35]:
models

{'FGLR': LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False)}

In [36]:
# Evaluate models on the validation set
def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    end = time()
    accuracy = round(accuracy_score(labels, pred), 3)
    precision = round(precision_score(labels, pred), 3)
    recall = round(recall_score(labels, pred), 3)
    print('{} -- Accuracy: {} / Precision: {} / Recall: {} / Latency: {}ms'.format(name, accuracy, precision, recall, round((end - start))))

In [37]:
for name, mdl in models.items():
    evaluate_model(name, mdl, val_features, val_labels)

FGLR -- Accuracy: 0.667 / Precision: 0.667 / Recall: 1.0 / Latency: 0ms
